<a href="https://colab.research.google.com/github/ShreyasFadnavis/Practice/blob/master/ProtoDash_extrtact_prototypes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install aix360
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 117.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 127.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 20.1 MB/s eta 0:00:0

In [ ]:
from datasets import load_dataset
from aix360.algorithms.protodash import ProtodashExplainer
import glob
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
import itertools

model_roberta = SentenceTransformer('all-distilroberta-v1')

In [ ]:
import pickle

# loads a list of sentences
with open ('psy_text', 'rb') as fp:
    plc_sent_list = pickle.load(fp)

## Runs the extraction of ProtoTypes

In [ ]:
def get_prototypes(sentences, num_prototyes=3):

    # define the number of prototypes
    m = num_prototyes

    # read the CSV file
    sentences = sentences

    # create vector per sentence
    docvec = model_roberta.encode(sentences)
    print(docvec.shape)

    sent_df = pd.DataFrame(sentences).squeeze()

    # initialize protodash
    explainer = ProtodashExplainer()
    docvec_sents = docvec[:len(sentences), :]

    with open('embeds.npy', 'wb') as f:
        np.save(f, docvec_sents)

    # call protodash explainer
    # S contains indices of the selected prototypes
    # W contains importance weights associated with the selected prototypes
    (W_sents, S_sents, _) = explainer.explain(docvec_sents,
                                              docvec_sents,
                                              m=m)

    # get prototypes from index
    df_sent_prototypes = sent_df.iloc[S_sents].copy()

    #normalize weights
    W_sents = np.around(W_sents/np.sum(W_sents), 2)

    print("Prototypes with weights:")
    print("----------------------------")
    for i in range(m):
        print('Sentence: '+str(i+1)+'=======================')
        print(W_sents[i], df_sent_prototypes.iloc[i])

    return W_sents, df_sent_prototypes

In [ ]:
weights_hc, hc_prototypes = get_prototypes(plc_sent_list[0],
                                           num_prototyes=10)

(538, 768)
     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -2.0000e+04  4e+00  1e+00  3e-01
 1:  1.6308e+00 -8.0322e+04  2e+01  1e+00  3e-01
 2:  1.0838e+00 -2.7977e+07  6e+03  1e+00  3e-01
 3:  7.4802e+07 -6.1119e+14  6e+14  6e-13  2e-05
 4:  7.4802e+07 -6.1120e+12  6e+12  6e-15  7e-06
 5:  7.4800e+07 -6.1197e+10  6e+10  2e-16  6e-08
 6:  7.4592e+07 -6.8862e+08  8e+08  1e-16  5e-10
 7:  5.9245e+07 -6.8117e+07  1e+08  1e-16  8e-11
 8:  5.7423e+06 -4.8890e+07  5e+07  1e-16  4e-12
 9:  1.6327e+06 -2.0787e+06  4e+06  2e-16  1e-12
10:  2.3808e+05 -2.6636e+05  5e+05  5e-17  3e-13
11:  3.4234e+04 -3.8159e+04  7e+04  7e-17  2e-13
12:  4.9040e+03 -5.4375e+03  1e+04  2e-16  5e-14
13:  6.9996e+02 -7.8095e+02  1e+03  2e-16  5e-15
14:  9.9127e+01 -1.1301e+02  2e+02  2e-16  3e-15
15:  1.3742e+01 -1.6648e+01  3e+01  1e-16  6e-15
16:  1.7822e+00 -2.5667e+00  4e+00  2e-16  2e-15
17:  1.7254e-01 -4.4492e-01  6e-01  1e-16  4e-16
18: -1.9933e-02 -1.0293e-01  8e-02  1e-16  2e-16
19: -3.561

/usr/local/lib/python3.10/dist-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/usr/local/lib/python3.10/dist-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/usr/local/lib/python3.10/dist-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/usr/local/lib/python3.10/dist-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:


10:  1.0234e+08 -9.0831e+08  1e+09  2e-16  1e-09
11:  3.0144e+07 -3.3764e+08  4e+08  1e-16  1e-08
12:  8.6756e+06 -1.6892e+07  3e+07  2e-16  6e-10
13:  1.3488e+06 -1.5318e+06  3e+06  1e-16  9e-12
14:  1.9411e+05 -2.1656e+05  4e+05  2e-16  6e-13
15:  2.7836e+04 -3.0815e+04  6e+04  2e-16  1e-13
16:  3.9823e+03 -4.4143e+03  8e+03  1e-16  6e-14
17:  5.6742e+02 -6.3532e+02  1e+03  2e-16  2e-14
18:  8.0030e+01 -9.2287e+01  2e+02  1e-16  1e-14
19:  1.0965e+01 -1.3718e+01  2e+01  2e-16  5e-15
20:  1.3637e+00 -2.1648e+00  4e+00  2e-16  2e-15
21:  9.9055e-02 -3.9828e-01  5e-01  2e-16  5e-16
22: -4.2589e-02 -1.0612e-01  6e-02  3e-16  2e-16
23: -5.2257e-02 -6.0140e-02  8e-03  3e-16  4e-17
24: -5.2755e-02 -5.3029e-02  3e-04  1e-16  4e-17
25: -5.2757e-02 -5.2760e-02  3e-06  1e-16  5e-17
26: -5.2757e-02 -5.2757e-02  3e-08  1e-16  4e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -8.0000e+04  2e+01  1e+00  5e-01
 1:  9.4072e+00 -3.6094e+05  7e+01  1e+00  5e-

In [ ]:
protos_10_hc = list(hc_prototypes)
protos_10_hc

['and so i was just like okay.',
 'yeah.',
 'i am like why me.',
 'she was my mom in that moment.',
 'we had different topics.',
 'so obviously he did that for over 10 years.',
 'and i wish i had a thought about sooner.',
 'i love him so much.',
 'i was driving through a backroad.',
 'at all.']

In [ ]:
plc_sent_list[0]

['they have been going okay.',
 'i have some goals that i am discovering and working towards achieving and yeah.',
 'damn it.',
 'yes.',
 'sometimes there are bad times.',
 'can i talk about my art.',
 'okay.',
 'well that is what i am doing.',
 'i am studying music in particular and i am a singersongwriter.',
 'i love to write music.',
 'actually i started writing i think i have got songs from when i was as old as 8yearsold.',
 'they are ridiculous but i love them very much.',
 'and yeah i actually recorded a song just last week and i uploaded it.',
 'it was not that good but i had actually decided this time to share it.',
 'so i write a lot of music.',
 'i do not share as much of it.',
 'my music has always been pretty private.',
 'but yeah.',
 'yeah.',
 'this time.',
 'i actually uploaded it.',
 'i have got a really old youtube channel.',
 'i think my first post was 2011 or 12.\nit was something really dumb.',
 'i am pretty sure it could have been it was revenge at my brother becaus

In [ ]:
weights_chr, chr_prototypes = get_prototypes(plc_sent_list[200], num_prototyes=10)

(283, 768)
     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -2.0000e+04  4e+00  1e+00  3e-01
 1:  1.4949e+00 -7.9810e+04  2e+01  1e+00  3e-01
 2:  1.0751e+00 -3.1543e+07  7e+03  1e+00  3e-01
 3:  7.5473e+07 -6.7703e+14  7e+14  6e-13  8e-06
 4:  7.5473e+07 -6.7703e+12  7e+12  6e-15  5e-06
 5:  7.5471e+07 -6.7783e+10  7e+10  1e-16  5e-08
 6:  7.5314e+07 -7.5696e+08  8e+08  1e-16  6e-10
 7:  6.3050e+07 -7.4579e+07  1e+08  2e-16  1e-10
 8:  4.0204e+06 -7.4384e+07  8e+07  2e-16  3e-12
 9:  1.5371e+06 -2.4350e+06  4e+06  2e-16  3e-12
10:  2.2733e+05 -2.4283e+05  5e+05  7e-17  4e-13
11:  3.2687e+04 -3.6689e+04  7e+04  8e-17  9e-14
12:  4.6831e+03 -5.1875e+03  1e+04  2e-16  2e-14
13:  6.6867e+02 -7.4535e+02  1e+03  3e-16  1e-14
14:  9.4785e+01 -1.0776e+02  2e+02  2e-16  1e-14
15:  1.3175e+01 -1.5841e+01  3e+01  2e-16  3e-15
16:  1.7238e+00 -2.4291e+00  4e+00  4e-16  5e-16
17:  1.7451e-01 -4.1567e-01  6e-01  2e-16  5e-16
18: -1.3838e-02 -9.3695e-02  8e-02  2e-16  2e-16
19: -3.029

In [ ]:
protos_10_chr = list(chr_prototypes)
protos_10_chr

['but also i received lots of support from acc and also the facilitated study group program.',
 'oh this is the  altruism condition.',
 'i think it is a good experience for me but i realized that my interest that is not lying in the realm of cell biology.',
 'and also there are more schools in ontario than british columbia.',
 'yeah.',
 'and i did not enjoy living with her that much back in high school and she asked me to apply for the programs i did not like.',
 'i try not to be the top student in the class.',
 'and also i have been living in the city for a long time.',
 'so throughout our practice as occupational therapist we measure and evaluate all these different dimensions and provide support and resources to clients to overcome the challenges in their daily lives.',
 'i think that is how i got the job.']